In [1]:
import sys, os
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot
from PyQt5.QtCore import Qt
from PyQt5.QtCore import QUrl
import joblib
import pretty_midi
import numpy as np
import pandas as pd
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent

In [2]:
label_list=np.load("label_list.npy")
matched_midi_arr=np.load("matched_midi_arr.npy", allow_pickle=True)

def make_prediction(clf, midi_path):
    features = get_features(midi_path)
    prediction_ind = list(clf.predict([features])[0]).index(1)
    prediction = label_list[prediction_ind]
    return prediction

def input_music(path):
    # Make a Prediction
    classifier = joblib.load('classifier_model')
    test_midi_path = path
    print(make_prediction(classifier, test_midi_path))
    ans=make_prediction(classifier, test_midi_path)
    return ans

def normalize_features(features):
    #This function normalizes the features to the range [-1, 1]
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    return [tempo, resolution, time_sig_1, time_sig_2]

def get_features(path):
    print(path)
    file = pretty_midi.PrettyMIDI(path)
            
    tempo = file.estimate_tempo()
    num_sig_changes = len(file.time_signature_changes)
    resolution = file.resolution
    ts_changes = file.time_signature_changes
    ts_1 = 4
    ts_2 = 4
    if len(ts_changes) > 0:
        ts_1 = ts_changes[0].numerator
        ts_2 = ts_changes[0].denominator
    return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])
"""
def pick_music(feature):
    df = pd.DataFrame(matched_midi_arr)
    df.columns =['Path', 'Genre']
    d_subset=df[df.Genre == feature]
    df_sel=d_subset.sample(5)
    print(df_sel)
"""

class MyWidget(QWidget):
    
    str="Jazz"
    
    def __init__(self, name='MyWidget'):
        super(MyWidget, self).__init__()
        self.setWindowTitle(name)
        self.cwd=os.getcwd()
        self.setGeometry(200,300,400,400)
        
        self.btn=QPushButton(self)
        self.btn.setText('Upload')
        self.btn.resize(self.btn.sizeHint())
        self.btn.move(100, 100)
        self.btn.clicked.connect(self.slot_btn_chooseFile)
        
        self.btn_2=QPushButton(self)
        self.btn_2.setText('Analyze')
        self.btn_2.resize(self.btn_2.sizeHint())
        self.btn_2.move(200, 100)
        self.btn_2.clicked.connect(self.slot_btn_open)
        
        self.label=QLabel(self)
        self.label.resize(300, 20)
        self.label.move(100, 50)
        
        self.show()
        
    def slot_btn_chooseFile(self):
        fileName_choose, filetype=QFileDialog.getOpenFileName(self, 'choose file', self.cwd, "Midi Files (*.mid)")
        if fileName_choose=="":
            print("\ncancel")
            return
        fileName_choose=os.path.basename(fileName_choose)
        self.label.setText(fileName_choose)
        self.str=input_music(fileName_choose)
        print("run+"+self.str)
        
    def slot_btn_open(self):
        self.recommend_widget=MyWidget_Rec(self.str)
        self.recommend_widget.show()
        #print("new")

In [3]:
class MyWidget_Rec(QWidget):
    
    genre=""
    
    def __init__(self, name='MyWidget_Rec'):
        super(MyWidget_Rec, self).__init__()
        self.setWindowTitle(name)
        self.setGeometry(650,300,400,400)
        
        self.genre=name
        
        self.label=QLabel(self)
        self.label.resize(300, 20)
        self.label.move(150, 50)
        self.label.setText(name)
        
        self.btn=QPushButton(self)
        self.btn.setText('Recommend')
        self.btn.resize(self.btn.sizeHint())
        self.btn.move(150, 100)
        self.btn.clicked.connect(self.slot_btn_playmusic)
        
        self.player=QMediaPlayer()
        
        #pick_music(name)
        
        self.show()
        
    def slot_btn_playmusic(self):
        
        str="recommend/"+self.genre+".mp3"
        full_file_path = os.path.join(os.getcwd(), str)
        url = QUrl.fromLocalFile(full_file_path)
        content = QMediaContent(url)

        self.player.setMedia(content)
        self.player.play()

        print("hello")

In [4]:
def main():
    app = QApplication(sys.argv)
    widget = MyWidget('Midi Classifier')
    widget.show()
    sys.exit(app.exec_())

if __name__=='__main__':
    main()

If_music_be_the_food_of_love.mid
International
If_music_be_the_food_of_love.mid
run+International


C:\Users\user\anaconda3\envs\midienv\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
C:\Users\user\anaconda3\envs\midienv\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


SystemExit: 0

C:\Users\user\anaconda3\envs\midienv\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
